In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [2]:
hotels=pd.read_csv("C:/Users/abdel/ACL/archive/hotels.csv")
reviews=pd.read_csv("C:/Users/abdel/ACL/archive/reviews.csv")
users=pd.read_csv("C:/Users/abdel/ACL/archive/users.csv")

In [3]:
print(f"Hotels duplicates: {hotels.duplicated().sum()}")
print(f"Reviews duplicates: {reviews.duplicated().sum()}")
print(f"Users duplicates: {users.duplicated().sum()}")
print("--------------------------------")
print(f"Hotels nuls: {hotels.isnull().sum()}")
print("--------------------------------")
print(f"Reviews nuls: {reviews.isnull().sum()}")
print("--------------------------------")
print(f"Users nuls: {users.isnull().sum()}")

Hotels duplicates: 0
Reviews duplicates: 0
Users duplicates: 0
--------------------------------
Hotels nuls: hotel_id                0
hotel_name              0
city                    0
country                 0
star_rating             0
lat                     0
lon                     0
cleanliness_base        0
comfort_base            0
facilities_base         0
location_base           0
staff_base              0
value_for_money_base    0
dtype: int64
--------------------------------
Reviews nuls: review_id                0
user_id                  0
hotel_id                 0
review_date              0
score_overall            0
score_cleanliness        0
score_comfort            0
score_facilities         0
score_location           0
score_staff              0
score_value_for_money    0
review_text              0
dtype: int64
--------------------------------
Users nuls: user_id           0
user_gender       0
country           0
age_group         0
traveller_type    0
join_date  

In [11]:
users.head()

,user_id,user_gender,country,age_group,traveller_type,join_date
0,1,Female,United Kingdom,35-44,Solo,2024-09-29
1,2,Male,United Kingdom,25-34,Solo,2023-11-29
2,3,Female,Mexico,25-34,Family,2022-04-03
3,4,Male,India,35-44,Family,2023-12-02
4,5,Other,Japan,25-34,Solo,2021-12-18


In [4]:
reviews.info()
print("--------------------------------")   
hotels.info()
print("--------------------------------")
users.info()
print("--------------------------------")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   review_id              50000 non-null  int64  
 1   user_id                50000 non-null  int64  
 2   hotel_id               50000 non-null  int64  
 3   review_date            50000 non-null  object 
 4   score_overall          50000 non-null  float64
 5   score_cleanliness      50000 non-null  float64
 6   score_comfort          50000 non-null  float64
 7   score_facilities       50000 non-null  float64
 8   score_location         50000 non-null  float64
 9   score_staff            50000 non-null  float64
 10  score_value_for_money  50000 non-null  float64
 11  review_text            50000 non-null  object 
dtypes: float64(7), int64(3), object(2)
memory usage: 4.6+ MB
--------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 2

This code merges the reviews, hotels, and users datasets into one DataFrame (df) by performing left joins on the hotel_id and user_id columns to combine review, hotel, and user information.

In [10]:
review_hotel_df=reviews.merge(hotels,on='hotel_id',how='left')
df=review_hotel_df.merge(users,on='user_id',how='left')
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   review_id              50000 non-null  int64  
 1   user_id                50000 non-null  int64  
 2   hotel_id               50000 non-null  int64  
 3   review_date            50000 non-null  object 
 4   score_overall          50000 non-null  float64
 5   score_cleanliness      50000 non-null  float64
 6   score_comfort          50000 non-null  float64
 7   score_facilities       50000 non-null  float64
 8   score_location         50000 non-null  float64
 9   score_staff            50000 non-null  float64
 10  score_value_for_money  50000 non-null  float64
 11  review_text            50000 non-null  object 
 12  hotel_name             50000 non-null  object 
 13  city                   50000 non-null  object 
 14  country_x              50000 non-null  object 
 15  st

After merging the hotels and users tables, the duplicate "country" columns were renamed to improve clarity: country_x (from hotels) became hotel_country, and country_y (from users) became user_country.

In [6]:
df.rename(columns={"country_x":"hotel_country","country_y":"user_country"},inplace=True)

The code assigns users' countries to geographic regions using a dictionary of regions and their countries. It adds a 'country_group' column with apply(), labeling unmatched countries as 'Other'

In [7]:
groups={'North_America':['United States','Canada'],
        'Western_Europe':['Germany','France','United Kingdom','Netherlands','Spain','Italy'],
        'Eastern_Europe':['Russia'],
        'East_Asia':['China','Japan','South Korea'],
        'Southeast_Asia':['Thailand','Singapore'],
        'Middle_East':['United Arab Emirates','Turkey'],
        'Africa':['Egypt','Nigeria','South Africa'],
        'Oceania':['Australia','New Zealand'],
        'South_America':['Brazil','Argentina'],
        'South_Asia':['India'],
        'North_America_Mexico':['Mexico']}

df["country_group"]=df["user_country"].apply(lambda x: next((key for key, value in groups.items() if x in value), "Other"))

df[["hotel_country","user_country","country_group"]]

,hotel_country,user_country,country_group
0,United States,New Zealand,Oceania
1,Japan,United Kingdom,Western_Europe
2,Egypt,Spain,Western_Europe
3,Spain,United Kingdom,Western_Europe
4,Russia,Australia,Oceania
...,...,...,...
49995,Nigeria,Mexico,North_America_Mexico
49996,Brazil,New Zealand,Oceania
49997,Singapore,Russia,Eastern_Europe
49998,Turkey,United Kingdom,Western_Europe


In [17]:
hotels["city"].unique()

array(['New York', 'London', 'Paris', 'Tokyo', 'Dubai', 'Singapore',
       'Sydney', 'Rio de Janeiro', 'Berlin', 'Toronto', 'Shanghai',
       'Mexico City', 'Mumbai', 'Rome', 'Cape Town', 'Seoul', 'Moscow',
       'Cairo', 'Barcelona', 'Bangkok', 'Istanbul', 'Amsterdam',
       'Buenos Aires', 'Lagos', 'Wellington'], dtype=object)